The AWS CLI doesn't support InvokeAgent.

In [ ]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

if os.path.dirname(find_dotenv()) not in sys.path:
    sys.path.append(os.path.dirname(find_dotenv()))

from src.utils import debug as debug_util

load_dotenv(override=True)

In [ ]:
import uuid
import boto3
from botocore.exceptions import ClientError

bedrock_agent_client = boto3.client(service_name="bedrock-agent", region_name=os.environ.get("AWS_DEFAULT_REGION"))
bedrock_agent_runtime_client = boto3.client(
    service_name="bedrock-agent-runtime", region_name=os.environ.get("AWS_DEFAULT_REGION")
)
agent_id = "R93VRMZXMC"
session_id = uuid.uuid4().hex

Prepare the agent before invoking it.

In [ ]:
try:
    response = bedrock_agent_client.get_agent(agentId=agent_id)
    agent = response["agent"]
    # debug_util.print_msg(str(agent), "Agent Info")

    if agent["agentStatus"] != "PREPARED":
        bedrock_agent_client.prepare_agent(agentId=agent_id)

    print(f"Agent {agent["agentArn"]} is ready!")
except ClientError as e:
    debug_util.print_msg(f"Agent operation error: {e}", color="red")

In [ ]:
question = "How can I create an EC2 instance?"

response = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId="TSTALIASID",
    sessionId=session_id,
    inputText=question,
)

In [ ]:
result = ""

for event in response.get("completion"):
    chunk = event["chunk"]
    result += chunk["bytes"].decode()

print(result)